In [ ]:
import socket
import json
import base64
import time
import os
import threading
timeout = 2.4 # change to low for low latency connections
debug = 1
if not debug:
    while 1:
        try:
            numThreadsPerDomain = int(input("Enter the number of Threads Per Domain: "))
            numObjectsPerThread = int(input("Enter the number of Objects Per Thread: "))
            harfile_name = input("Enter the HAR File Name: ")
            harfile = open(harfile_name,"r")
            # pipe = int(input("Enter 1 for Pipelined Execution and 0 for not: "))
            pipe = 0
            PIPELINED = False if pipe ==1 else False
            break
        except:
            print("Invalid input. Please try again.")
else:
    numThreadsPerDomain = 3
    numObjectsPerThread = 2
    harfile_name = "indianexpress.har"
    harfile = open(harfile_name,"r")
    PIPELINED = False

foldername = '.'.join(harfile_name.split(".")[0:-1])
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])
domain_dict = {}
act = threading.activeCount()
def fetch_thread(request_small):
    if not PIPELINED:
        for i in request_small:
#             print(threading.current_thread().name, " Processed url ", i["request"]["url"])
            url = i["request"]["url"]
            prefix = url[0:5]
            # print(url)
            if prefix != "http:":
#                 print("skipped ", url)
                continue
            url_split = url.split("://")

            content_file_path = foldername
            content_file_path = content_file_path + "/" + url_split[-1]
            if 1==1:#int(i["response"]["content"]["size"])!=0:
                har_req = i["request"]
                http_version = "HTTP" + har_req["httpVersion"][4:]
                request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
                hostname = (har_req["url"].split("//")[1]).split("/")[0]
                request_str = request_str + "Host"+": " + hostname+"\n"
                for el in har_req["headers"]:
                    if el["name"] not in ("User-Agent","Connection"):continue
                    request_str = request_str + el["name"]+": " + el["value"]+"\n"
                request_str = request_str + "\n"
                    # print("------------------------------")
                    # print(request_str)
                    # print("------------------------------")
                encoded_request = request_str.encode()
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
            #   print("Hostname is ", hostname)
                client_socket.connect((hostname, 80))
                client_socket.send(encoded_request)
                received_bytes = b""
                client_socket.settimeout(timeout)
                while 1:
                    try:
                        msg = client_socket.recv(1024)
                        if(len(msg)==0):
                            break
                        received_bytes = received_bytes + msg
                    except:
                        break
                split = received_bytes.split(b"\r\n\r\n")
                if len(received_bytes)<1 or len(split)<2:
                    # print("unsplit ", url, "byter r ", received_bytes)
                    continue
                client_socket.close()
                if(len(split[0])<8):
                    header = split[1]
                    content = split[2]
                else:
                    header = split[0]
                    content = split[1]
                # print("c is   "  ,foldername, " ",content_file_path)
                if content_file_path[-1]=="/":
                    print("Wierd   ", url, "rec ",received_bytes )
                    
                    content_file_path = content_file_path + "index.html"
                if len(content_file_path)>90:
                    # print("Truncated a name")
                    content_file_path = content_file_path[:90]
                if not os.path.exists(os.path.dirname(content_file_path)):
                    try:
                        os.makedirs(os.path.dirname(content_file_path))
                    except: # Guard against race condition
                        print("Error :Race")                
                headerfile = open(content_file_path+"_header","wb")
                headerfile.write(header)
                headerfile.close()
#                 print("Processed ",content_file_path)
                contentfile = open(content_file_path,"wb")
                contentfile.write(content)
                contentfile.close()
#         else:
#             for i in request_small:
                
def dom_thread_fun(reqlist):
    tl = []
    done = 0
    tind = 0
    while len(reqlist) != done:
        while len(tl) >= numThreadsPerDomain:
            for th in tl:
                if not th.isAlive():
                    tl.remove(th)
                    break
        tempList = reqlist[done:min(done+numObjectsPerThread, len(reqlist))]
        done += len(tempList)
        nt  = threading.Thread(name=threading.current_thread().name+"fetch"+str(tind),target=fetch_thread,args=(tempList,))
        tl.append(nt)
        tind+=1
        nt.start()
start_time = time.time()
for i in har_json["log"]["entries"]:
    domain_name = i["request"]["url"].split("//",maxsplit=1)[-1].split("/")[0].split(".",maxsplit=1)[1]
    if domain_name in domain_dict.keys():
        domain_dict[domain_name].append(i)
    else:
        domain_dict[domain_name] = [i]
list_of_threads = []
for dom in domain_dict.keys():
    t = threading.Thread(name=dom,target=dom_thread_fun,args=(domain_dict[dom],))
    list_of_threads.append(t)
    t.start()
while(threading.activeCount()!=act):
#     print("-")
    pass
elapsed_time = time.time() - start_time
print("Time Elasped is ", elapsed_time)

Processed  indianexpress/images.indianexpress.com/2017/09/pitru-paksha-2017-620x400.jpg?w=210
Processed  indianexpress/images.indianexpress.com/2017/09/varun-dhawan-7597.jpg?w=210
Processed  indianexpress/images.indianexpress.com/2017/07/deepika-padukone11.jpg?w=210
Wierd    

In [ ]:
import socket
import json
import base64
import time
import os
import threading

debug = 1
if not debug:
    while 1:
        try:
            numThreadsPerDomain = int(input("Enter the number of Threads Per Domain: "))
            numObjectsPerThread = int(input("Enter the number of Objects Per Thread: "))
            harfile_name = input("Enter the HAR File Name: ")
            harfile = open(harfile_name,"r")
            # pipe = int(input("Enter 1 for Pipelined Execution and 0 for not: "))
            pipe = 0
            PIPELINED = False if pipe ==1 else False
            break
        except:
            print("Invalid input. Please try again.")
else:
    numThreadsPerDomain = 3
    numObjectsPerThread = 2
    harfile_name = "indianexpress.har"
    harfile = open(harfile_name,"r")
    PIPELINED = False

foldername = harfile_name.split(".")[0]
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])
domain_dict = {}
act = threading.activeCount()

In [ ]:
num_requests

In [ ]:
har_json["log"]["entries"][0]

In [ ]:
request_small = [har_json["log"]["entries"][0]]

In [ ]:
i = request_small[0]

In [ ]:
            url = i["request"]["url"]
            prefix = url[0:5]
            # print(url)
            url_split = url.split("://")

            content_file_path = foldername
            content_file_path = content_file_path + "/" + url_split[-1]
            if 1==1:#int(i["response"]["content"]["size"])!=0:
                har_req = i["request"]
                http_version = "HTTP" + har_req["httpVersion"][4:]
                request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
                hostname = (har_req["url"].split("//")[1]).split("/")[0]
                request_str = request_str + "Host"+": " + hostname+"\n"
                for el in har_req["headers"]:
                    if el["name"] not in ("User-Agent","Connection"):continue
                    request_str = request_str + el["name"]+": " + el["value"]+"\n"
                request_str = request_str + "\n"
                    # print("------------------------------")
                    # print(request_str)
                    # print("------------------------------")
                encoded_request = request_str.encode()
                

In [ ]:
request_str

In [ ]:
hostname

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
#   print("Hostname is ", hostname)
client_socket.connect((hostname, 80))
client_socket.send(encoded_request)
received_bytes = b""
client_socket.settimeout(1.0)
while 1:
    try:
        msg = client_socket.recv(1024)
        if(len(msg)==0):
            break
        received_bytes = received_bytes + msg
    except:
        break


In [ ]:
received_bytes

In [ ]:
                split = received_bytes.split(b"\r\n\r\n")
                if len(received_bytes)<1 or len(split)<2:
                    # print("unsplit ", url, "byter r ", received_bytes)
                    print("bbbb")
                client_socket.close()
                if(len(split[0])<8):
                    header = split[1]
                    content = split[2]
                else:
                    header = split[0]
                    content = split[1]
                # print("c is   "  ,foldername, " ",content_file_path)


In [ ]:
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
            #   print("Hostname is ", hostname)
                client_socket.connect((hostname, 80))
                client_socket.send(encoded_request)
                received_bytes = b""
                client_socket.settimeout(0.7)
                while 1:
                    try:
                        msg = client_socket.recv(1024)
                        if(len(msg)==0):
                            break
                        received_bytes = received_bytes + msg
                    except:
                        break
                split = received_bytes.split(b"\r\n\r\n")
                if len(received_bytes)<1 or len(split)<2:
                    # print("unsplit ", url, "byter r ", received_bytes)
                    continue
                client_socket.close()
                if(len(split[0])<8):
                    header = split[1]
                    content = split[2]
                else:
                    header = split[0]
                    content = split[1]
                # print("c is   "  ,foldername, " ",content_file_path)
                if content_file_path[-1]=="/":
                    content_file_path = content_file_path + "index.html"
                if len(content_file_path)>90:
                    # print("Truncated a name")
                    content_file_path = content_file_path[:90]
                if not os.path.exists(os.path.dirname(content_file_path)):
                    try:
                        os.makedirs(os.path.dirname(content_file_path))
                    except: # Guard against race condition
                        print("Error :Race")                
                headerfile = open(content_file_path+"_header","wb")
                headerfile.write(header)
                headerfile.close()
                print("Processed ",content_file_path)
                contentfile = open(content_file_path,"wb")
                contentfile.write(content)
                contentfile.close()
#         else:
#             for i in request_small:

In [ ]:
def fetch_thread(request_small):
    if not PIPELINED:
        for i in request_small:
#             print(threading.current_thread().name, " Processed url ", i["request"]["url"])
            url = i["request"]["url"]
            prefix = url[0:5]
            # print(url)
            if prefix != "http:":
                print("skipped ", url)
                continue
            url_split = url.split("://")

            content_file_path = foldername
            content_file_path = content_file_path + "/" + url_split[-1]
            if 1==1:#int(i["response"]["content"]["size"])!=0:
                har_req = i["request"]
                http_version = "HTTP" + har_req["httpVersion"][4:]
                request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
                hostname = (har_req["url"].split("//")[1]).split("/")[0]
                request_str = request_str + "Host"+": " + hostname+"\n"
                for el in har_req["headers"]:
                    if el["name"] not in ("User-Agent","Connection"):continue
                    request_str = request_str + el["name"]+": " + el["value"]+"\n"
                request_str = request_str + "\n"
                    # print("------------------------------")
                    # print(request_str)
                    # print("------------------------------")
                encoded_request = request_str.encode()
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
            #   print("Hostname is ", hostname)
                client_socket.connect((hostname, 80))
                client_socket.send(encoded_request)
                received_bytes = b""
                client_socket.settimeout(0.7)
                while 1:
                    try:
                        msg = client_socket.recv(1024)
                        if(len(msg)==0):
                            break
                        received_bytes = received_bytes + msg
                    except:
                        break
                split = received_bytes.split(b"\r\n\r\n")
                if len(received_bytes)<1 or len(split)<2:
                    # print("unsplit ", url, "byter r ", received_bytes)
                    continue
                client_socket.close()
                if(len(split[0])<8):
                    header = split[1]
                    content = split[2]
                else:
                    header = split[0]
                    content = split[1]
                # print("c is   "  ,foldername, " ",content_file_path)
                if content_file_path[-1]=="/":
                    content_file_path = content_file_path + "index.html"
                if len(content_file_path)>90:
                    # print("Truncated a name")
                    content_file_path = content_file_path[:90]
                if not os.path.exists(os.path.dirname(content_file_path)):
                    try:
                        os.makedirs(os.path.dirname(content_file_path))
                    except: # Guard against race condition
                        print("Error :Race")                
                headerfile = open(content_file_path+"_header","wb")
                headerfile.write(header)
                headerfile.close()
                print("Processed ",content_file_path)
                contentfile = open(content_file_path,"wb")
                contentfile.write(content)
                contentfile.close()
#         else:
#             for i in request_small:
                
def dom_thread_fun(reqlist):
    tl = []
    done = 0
    tind = 0
    while len(reqlist) != done:
        while len(tl) >= numThreadsPerDomain:
            for th in tl:
                if not th.isAlive():
                    tl.remove(th)
                    break
        tempList = reqlist[done:min(done+numObjectsPerThread, len(reqlist))]
        done += len(tempList)
        nt  = threading.Thread(name=threading.current_thread().name+"fetch"+str(tind),target=fetch_thread,args=(tempList,))
        tl.append(nt)
        tind+=1
        nt.start()
start_time = time.time()

In [ ]:
for i in har_json["log"]["entries"]:
    domain_name = i["request"]["url"].split("//",maxsplit=1)[-1].split("/")[0].split(".",maxsplit=1)[1]
    if domain_name in domain_dict.keys():
        domain_dict[domain_name].append(i)
    else:
        domain_dict[domain_name] = [i]
list_of_threads = []

In [ ]:
domain_dict

In [ ]:
for dom in domain_dict.keys():
    t = threading.Thread(name=dom,target=dom_thread_fun,args=(domain_dict[dom],))
    list_of_threads.append(t)
    t.start()
while(threading.activeCount()!=act):
#     print("-")
    pass
elapsed_time = time.time() - start_time
print("Time Elasped is ", elapsed_time)

In [ ]:
import socket
import json
import base64
import time
import os
import threading
numThreadsPerDomain = 3
numObjectsPerThread = 2
harfile_name = "bbc.har"
PIPELINED = False
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])
domain_dict = {}
act = threading.activeCount()


In [ ]:
a = {}

In [ ]:
l = ["a","b","c"]

In [ ]:
a[elem] = 0 if elem not in a.keys()  else a[elem]+1

In [ ]:
a

In [ ]:
p = []

In [ ]:
har_json["log"]["entries"][2]

In [ ]:
for i in range(len(har_json["log"]["entries"])):
    dd =  har_json["log"]["entries"][i]["response"]["content"]["size"]
    if dd==0:
        p.append(har_json["log"]["entries"][i])
    print(har_json["log"]["entries"][i]["response"]["content"]["size"])

In [ ]:
p[2]

In [ ]:
dd = "http://wwwdefdfw.gsta.hbj.tic.com/ds./././.d/saddar"

In [ ]:
dd.split("//")[1].split("/",maxsplit=1)[0].split(".",maxsplit=1)[-1]

In [ ]:
len(har_json["log"]["entries"])

In [ ]:
num_requests

In [ ]:
def fetch_thread(request_small):
    if not PIPELINED:
        for i in request_small:
#             print(threading.current_thread().name, " Processed url ", i["request"]["url"])
            url = i["request"]["url"]
            prefix = url[0:5]
            if prefix != "http:":
                continue
            url_split = url.split("://")

            content_file_path = foldername
            content_file_path = content_file_path + "/" + url_split[-1]
            if int(i["response"]["content"]["size"])!=0:
                if content_file_path[-1]=="/":
                    content_file_path = content_file_path + "index.html"
                if not os.path.exists(os.path.dirname(content_file_path)):
                    try:
                        os.makedirs(os.path.dirname(content_file_path))
                    except: # Guard against race condition
                        print("Error :Race")
                        exit(1)
                har_req = i["request"]
                http_version = "HTTP" + har_req["httpVersion"][4:]
                request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
                hostname = (har_req["url"].split("//")[1]).split("/")[0]
                request_str = request_str + "Host"+": " + hostname+"\n"
                for el in har_req["headers"]:
                    if el["name"] not in ("User-Agent","Connection"):continue
                    request_str = request_str + el["name"]+": " + el["value"]+"\n"
                request_str = request_str + "\n"
        #         print("------------------------------")
        #         print(request_str)
        #         print("------------------------------")
                encoded_request = request_str.encode()
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
                print("Hostname is ", hostname)
                client_socket.connect((hostname, 80))
                client_socket.send(encoded_request)
                received_bytes = b""
                client_socket.settimeout(0.2)
                while 1:
                    try:
                        msg = client_socket.recv(1024)
                        if(len(msg)==0):
                            break
                        received_bytes = received_bytes + msg
                    except:
                        break

                header = received_bytes.split(b"\r\n\r\n")[0]
                client_socket.close()
                content = received_bytes.split(b"\r\n\r\n")[-1]
                headerfile = open(content_file_path+"_header","wb")
                headerfile.write(header)
                headerfile.close()
                contentfile = open(content_file_path,"wb")
                contentfile.write(content)
                contentfile.close()
#         else:
#             for i in request_small:
                


In [ ]:
def dom_thread_fun(reqlist):
    tl = []
    done = 0
    tind = 0
    while len(reqlist) != done:
        if len(tl) == numThreadsPerDomain:
            while 1:
                a = 0
                for th in tl:
                    if th.isAlive():
                        a = 1
                if a==1:
                    continue
                else:
                    tl =[]
                    break
        tempList = reqlist[done:min(done+numObjectsPerThread, len(reqlist))]
        done += len(tempList)
        nt  = threading.Thread(name=threading.current_thread().name+"fetch"+str(tind),target=fetch_thread,args=(tempList,))
        tl.append(nt)
        tind+=1
        nt.start()

In [ ]:
start_time = time.time()
for i in har_json["log"]["entries"]:
    domain_name = i["request"]["url"].split("//",maxsplit=1)[-1].split("/")[0].split(".",maxsplit=1)[1]
    if domain_name in domain_dict.keys():
        domain_dict[domain_name].append(i)
    else:
        domain_dict[domain_name] = [i]
list_of_threads = []
for dom in domain_dict.keys():
    t = threading.Thread(name=dom,target=dom_thread_fun,args=(domain_dict[dom],))
    list_of_threads.append(t)
    t.start()
while(threading.activeCount()!=act):
#     print("-")
    pass
elapsed_time = time.time() - start_time
print("Time Elasped is ", elapsed_time)

In [ ]:
import socket
import json
import base64
import time
import os
import threading
numThreadsPerDomain = 3
numObjectsPerThread = 2
harfile_name = "bbc.har"
PIPELINED = False
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])
domain_dict = {}
act = threading.activeCount()

In [ ]:
num_requests

In [ ]:
request_small = har_json["log"]["entries"]

In [ ]:
har_json= json.load(open("bbc.har","r"))
for i in har_json["log"]["entries"]:
    domain_name = i["request"]["url"].split("//",maxsplit=1)[-1].split("/")[0].split(".",maxsplit=1)[1]
    if domain_name in domain_dict.keys():
        domain_dict[domain_name].append(i)
    else:
        domain_dict[domain_name] = [i]
request_small = domain_dict["bbci.co.uk"]
request_small = request_small[0:6]

In [ ]:
domain_dict.keys()

In [ ]:
len(request_small)

In [ ]:
        for i in request_small:
#             print(threading.current_thread().name, " Processed url ", i["request"]["url"])
            url = i["request"]["url"]
            prefix = url[0:5]
            if prefix != "http:":
                continue
            url_split = url.split("://")

            content_file_path = foldername
            content_file_path = content_file_path + "/" + url_split[-1]
            if 1==1:
                if content_file_path[-1]=="/":
                    content_file_path = content_file_path + "index.html"
                if not os.path.exists(os.path.dirname(content_file_path)):
                    try:
                        os.makedirs(os.path.dirname(content_file_path))
                    except: # Guard against race condition
                        print("Error :Race")
                        exit(1)
                har_req = i["request"]
                http_version = "HTTP" + har_req["httpVersion"][4:]
                request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
                hostname = (har_req["url"].split("//")[1]).split("/")[0]
                request_str = request_str + "Host"+": " + hostname+"\n"
                for el in har_req["headers"]:
                    if el["name"] not in ("User-Agent","Connection"):continue
                    request_str = request_str + el["name"]+": " + el["value"]+"\n"
                request_str = request_str + "\r\n"
#                 print("------------------------------")
#                 print(request_str)
#                 print("------------------------------")
                encoded_request = request_str.encode()
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
                print("Hostname is ", hostname)
                client_socket.connect((hostname, 80))
                client_socket.send(encoded_request)
                received_bytes = b""
                client_socket.settimeout(0.6)
                while 1:
                    try:
                        msg = client_socket.recv(1024)
                        print(len(msg))
                        if(len(msg)==0):
                            break
                        received_bytes = received_bytes + msg
                    except:
                        break
                spl = received_bytes.split(b"\r\n\r\n")
#                 print(spl)
                header = spl[0]
                client_socket.close()
                content = spl[1]
                headerfile = open(content_file_path+"_header","wb")
                headerfile.write(header)
                headerfile.close()
                contentfile = open(content_file_path,"wb")
#                 print(content)
                contentfile.write(content)
                contentfile.close()


In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
x = 0
for i in request_small:
#             print(threading.current_thread().name, " Processed url ", i["request"]["url"])
    url = i["request"]["url"]
    prefix = url[0:5]
    if prefix != "http:":
        continue
    url_split = url.split("://")

    content_file_path = foldername
    content_file_path = content_file_path + "/" + url_split[-1]
    if 1:
        if content_file_path[-1]=="/":
            content_file_path = content_file_path + "index.html"
        if not os.path.exists(os.path.dirname(content_file_path)):
            try:
                os.makedirs(os.path.dirname(content_file_path))
            except: # Guard against race condition
                print("Error :Race")
                exit(1)
        har_req = i["request"]
        http_version = "HTTP" + har_req["httpVersion"][4:]
        request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
        hostname = (har_req["url"].split("//")[1]).split("/")[0]
        request_str = request_str + "Host"+": " + hostname+"\n"
        request_str = request_str + "Connection"+": " + "keep-alive"+"\n"
        for el in har_req["headers"]:
            if el["name"] not in ("User-Agent"):continue
            request_str = request_str + el["name"]+": " + el["value"]+"\n"
        request_str = request_str + "\r\n\r\n"
        print("------------------------------")
        print(request_str)
        print("------------------------------")
        encoded_request = request_str.encode()
        print("Hostname is ", hostname)
        if x==0:
            x=1
            client_socket.connect((hostname, 80))
        client_socket.send(encoded_request)

In [ ]:
received_bytes = b""
client_socket.settimeout(0.6)
while 1:
    try:
        msg = client_socket.recv(1024)
        print(len(msg))
        if(len(msg)==0):
            pass
        received_bytes = received_bytes + msg
    except:
        print("eeeee")
        break
spl = received_bytes.split(b"\r\n\r\n")

In [ ]:
len(spl)

In [ ]:
spl[0]

In [ ]:
spl[1]

In [ ]:
for n in spl:
    print("--------------------")
    print(n[0:min(len(n),150)])

In [ ]:
print(received_bytes)

In [ ]:
import socket
import json
import base64
import os
import threading

harfile_name = "bbc.har"
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])
for i in har_json["log"]["entries"]:
    url = i["request"]["url"]
    prefix = url[0:5]
    # print(prefix)
    if prefix != "http:":
        continue
    url_split = url.split("://")
    
    content_file_path = foldername
    content_file_path = content_file_path + "/" + url_split[-1]
    if int(i["response"]["content"]["size"])!=0:
#         if not "text" in i["response"]["content"].keys():
#             continue
        print(content_file_path)
        if content_file_path[-1]=="/":
            content_file_path = content_file_path + "index.html"
        # print(os.path.dirname(right_part))
        if not os.path.exists(os.path.dirname(content_file_path)):
            try:
                os.makedirs(os.path.dirname(content_file_path))
            except: # Guard against race condition
                print("Error :Race")
                exit(1)
        har_req = i["request"]
        http_version = "HTTP" + har_req["httpVersion"][4:]
        request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
        hostname = (har_req["url"].split("//")[1]).split("/")[0]
        request_str = request_str + "Host"+": " + hostname+"\n"
        for el in har_req["headers"]:
            if el["name"] not in ("User-Agent","Connection"):continue
            request_str = request_str + el["name"]+": " + el["value"]+"\n"
#         request_str = request_str + "Conncetion"+": " + "Close"+"\n"
#         request_str = request_str + "User-Agent"+": " + har_req["headers"]+"\n"
        request_str = request_str + "\n"
        print("------------------------------")
        print(request_str)
        print("------------------------------")
        encoded_request = request_str.encode()
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
        print("Hostname is ", hostname)
        client_socket.connect((hostname, 80))
        client_socket.send(encoded_request)
        received_bytes = b""
        client_socket.settimeout(0.2)
        while 1:
            try:
                msg = client_socket.recv(1024)
                if(len(msg)==0):
                    break
                received_bytes = received_bytes + msg
            except:
                break
                
        header = received_bytes.split(b"\r\n\r\n")[0]
        client_socket.close()
        content = received_bytes.split(b"\r\n\r\n")[-1]
        headerfile = open(content_file_path+"_header","wb")
        headerfile.write(header)
        headerfile.close()
        contentfile = open(content_file_path,"wb")
        contentfile.write(content)
        contentfile.close()
        
        
        


In [ ]:
import socket
import json
import base64
import os
harfile_name = "bbc.har"
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])

  

In [ ]:
c = har_json["log"]["entries"][11]

In [ ]:
c

In [ ]:
c

In [ ]:
r = c["request"]

In [ ]:
r

In [ ]:
requ = r["method"] + " /" + r["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ r["httpVersion"] + "\n" 

In [ ]:
requ

In [ ]:
url


In [ ]:
requ = requ[:-9] + "HTTP/1.1\n"

In [ ]:
r["url"]

In [ ]:
hostname = (r["url"].split("//")[1]).split("/")[0]

In [ ]:
hostname

In [ ]:
crauy.split(",",maxsplit=1)

In [ ]:
for el in r["headers"]:
    if el["name"] not in ("User-Agent","Connection"):continue
#     if(el["name"]=="Host"):
#         hostname = el["value"]

    requ = requ + el["name"]+": " + el["value"]+"\n"

In [ ]:
requ = requ + "Host"+": " + hostname+"\n"

In [ ]:
requ

In [ ]:
requ = requ+"\n"

In [ ]:
en = requ.encode()

In [ ]:
hostname

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
client_socket.connect((hostname, 80))                               


In [ ]:
client_socket.send(en)


In [ ]:
d = b""

In [ ]:
import re
# substitute sequence of non-ASCII characters with single space
# str = re.sub(r'[^\x00-\x7F]+',' ', str)


In [ ]:
while 1:
    msg = client_socket.recv(1024)
    if len(msg)!=1024:
        d = d + msg
        break
    else:
#         msg = re.sub(r'[^\x00-\x7F]+',' ', msg)
        d= d+msg


In [ ]:
len(d)

In [ ]:
d.split(b"\r\n\r\n")[-1]

In [ ]:
d[:777]

In [ ]:
dec = d.decode("utf8")

In [ ]:
dec

In [ ]:
dec[-100:]

In [ ]:
    msg = client_socket.recv(1024)


In [ ]:
    msg = client_socket.recv(4096)
    d= d+msg
#     if not msg:
#         break
#     else:
# #         msg = re.sub(r'[^\x00-\x7F]+',' ', msg)
#         d= d+msg

In [ ]:
msg

In [ ]:
len(msg)

In [ ]:
type(msg)

In [ ]:
print(msg.decode("utf8"))

In [ ]:
d[-100:]

In [ ]:
for dddd in range(190):
    msg = client_socket.recv(1024)
    if not msg:
        break
    else:
#         msg = re.sub(r'[^\x00-\x7F]+',' ', msg)
        d= d+msg.decode()


In [ ]:
len(d)

In [ ]:
print(d[-1300:])

In [ ]:
len(d)

In [ ]:
d [-300:]

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
client_socket.connect((hostname, 80))                               

client_socket.send(en)
msg = client_socket.recv(50000)
client_socket.close()
decod = msg.decode('ascii')
print (decod)

In [ ]:
print(decod[-200:-1])

In [ ]:
len(decod)

In [ ]:

# client_socket.close()

# print (msg.decode('ascii'))


In [ ]:
for i in har_json["log"]["entries"]:
    url = i["request"]["url"]
    prefix = url[0:5]

In [ ]:
i

In [ ]:
import json

In [ ]:
a = open("bbc.har")

In [ ]:
a = json.load(a)

In [ ]:
a["log"].keys()

In [ ]:
a["log"]["pages"]

In [ ]:
len(a["log"]["entries"])

In [ ]:
a["log"]["entries"][0]

In [ ]:
a["log"]["entries"][11]

In [ ]:
a["log"]["entries"][11].keys()

In [ ]:
type(a["log"]["entries"][11]["response"]["content"]["text"])

In [ ]:
 s = a["log"]["entries"][11]["response"]["content"]["text"]

In [ ]:
s.decode('base64')

In [ ]:
import base64
coded_string = s
dec = base64.b64decode(coded_string)

In [ ]:
f = open("out.jpg","wb")

In [ ]:
f.write(dec)

In [ ]:
f.close()